# Level up your Pandas with `df.query` and `df.eval`
10/25/2023

<br>
<br>
<br>

This notebook introduces students to `df.query` and `df.eval`. These are some reasons why you may want to learn these functions:
- You may prefer this syntax, particularly `df.query` if you have a background in SQL.
- There may be some [efficiency gains](https://jakevdp.github.io/PythonDataScienceHandbook/03.12-performance-eval-and-query.html).
- You may encounter this syntax in other people's code, so it's important to be familiar with it.

<br>
<br>

This notebook assumes that students are already familiar with basic Python and Pandas. *The code cells include lots of comments and links for you to better understand the code or learn more about it. The end of the notebook also includes resources for further learning.*

<br>
<br>
<br>

## Importing libraries that we're going to use

In [ ]:
# Main library for this tutorial, which includes df.query and df.eval
import pandas as pd
# Using for nan and sort
import numpy as np
# Using for data for exercises
from sklearn.datasets import load_iris

<br>
<br>
<br>

## Loading and cleaning data *for the examples*

**You don't need to download these data** (although you can if you want). **The exercises will use different data.**

[2022 National Health Interview Survey](https://www.cdc.gov/nchs/nhis/2022nhis.htm)
- [Summary](https://ftp.cdc.gov/pub/Health_Statistics/NCHS/Dataset_Documentation/NHIS/2022/adult-summary.pdf)
- [Codebook](https://ftp.cdc.gov/pub/Health_Statistics/NCHS/Dataset_Documentation/NHIS/2022/adult-codebook.pdf)
- [URL **that automatically downloads the data TO YOUR COMPUTER**](https://ftp.cdc.gov/pub/Health_Statistics/NCHS/Datasets/NHIS/2022/adult22csv.zip)
<br>
<br>

These data will allow us to (non-exhaustively and non-rigorously) explore some questions about meditation:
- What percentage of American adults meditate?
- What percentage of college-age American adults meditate?
- How about in large metropolitan areas?
- Does it depend on the number of people living in the household?

<br>

Side note: meditation has various [benefits](https://www.mayoclinic.org/tests-procedures/meditation/in-depth/meditation/art-20045858). If you're interested to learn, there's [meditation at Northwestern](https://www.northwestern.edu/religious-life/explore-our-programs/mindfulness/), including [guided meditations on the go](https://www.northwestern.edu/breathe/).

In [ ]:
# Loading the data into this notebook (not your computer) from GitHub
# https://www.geeksforgeeks.org/ways-to-import-csv-files-in-google-colab/
# https://github.com/orgs/community/discussions/22537
url_data = 'https://raw.githubusercontent.com/emiliolehoucq/trainings/main/data/nhis_2022_data.csv'
df = pd.read_csv(url_data)

# Making sure the data looks ok
df.head()

We're recoding or dropping some values for the sake of clarity and convenience.

This is not part of the material for this notebook, so don't worry if you don't understand everything. You can always come back to this part on your own if you want to practice reading and making sense of Pandas code.

In [ ]:
"""
Variables about meditation that we're going to use:

MEDITATE_A
Meditation includes Mindfulness, Mantra, and Spiritual meditation. In meditation a person focuses, stills, or quiets the mind. During the past 12 months, did you use any of these types of meditation?
Asked if HHSTAT_A = 1 (Indicates person is the Sample Adult)
1 = Yes
2 = No
7 = Refused
8 = Not Ascertained
9 = Don't know

MEDIHLTH_A
During the past 12 months, did you use meditation ... to restore your overall health?
Asked if HHSTAT_A = 1 and MEDITATE_A = 1
1 = Yes
2 = No
7 = Refused
8 = Not Ascertained
9 = Don't know

YOGAMED_A
Did you do meditation as part of Yoga?
Asked if HHSTAT_A = 1 and YOGA_A = 1
1 = Yes
2 = No
7 = Refused
8 = Not Ascertained
9 = Don't know
"""

# Iterating over columns of interest to replace values
# This would be useful if you were dealing with a larger number of similar columns

# Creating list to store names and descriptions of columns of interest
columns_of_interest = ['MEDITATE_A', 'MEDIHLTH_A', 'YOGAMED_A']

# Creating lists to store original and replacement values
original_values = [1, 2, 7, 8, 9]
replacement_values = ['Yes', 'No', np.nan, np.nan, np.nan]

# Iterating replacing values
for column_name in columns_of_interest:
  # https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html
  df[column_name].replace(original_values, replacement_values, inplace = True)

# Replacing values for other columns

# 2013 NCHS Urban-Rural Classification Scheme for Counties
# 1 = Large central metro
# 2 = Large fringe metro
# 3 = Medium and small metro
# 4 = Nonmetropolitan
# Note: "Large" = 1M or more
df['URBRRL'].replace(
    [1, 2, 3, 4],
    ['Large central metro', 'Large fringe metro', 'Medium and small metro', 'Nonmetropolitan'],
    inplace = True
)

# Age of SA (top coded)
# Asked if HHSTAT_A = 1
# 18 - 84: age
# 85 = 85+ years
# 97 = refused
# 98 = Not ascertained
# 99 = Don't know
df[df['AGEP_A'] > 84] = np.nan

# Number of adults in Sample Adult family, top-coded 3+
# Asked if HHSTAT_A = 1
# 1 = 1 adult
# 2 = 2 adult
# 3 = 3+ adult
# 8 = Not ascertained
df[df['PCNTADLT_A'] > 2] = np.nan

# Number of children in Sample Adult family, top-coded 3+
# Asked if HHSTAT_A = 1
# 0 = 0 children
# 1 = 1 child
# 2 = 2 children
# 3 = 3+ children
# 8 = Not ascertained
df[df['PCNTKIDS_A'] > 2] = np.nan

<br>
<br>
<br>

## Loading and cleaning data *for the exercises*

For the exercises we're using the [Iris plants dataset from Scikit-Learn](https://scikit-learn.org/stable/datasets/toy_dataset.html#iris-plants-dataset).

<br>
<br>

**Please do run the cell below so that you can do the exercises later.** Again, no worries if you don't understand everything, since it's not part of the materials for this notebook. Feel free to come back later on your own.

<br>
<br>

This is the way to load and clean the data (for this notebook, the key aspect is that `df.query` seems to not be able to handle column names with parentheses):

In [ ]:
# Loading the data
# https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html#sklearn.datasets.load_iris
df_exercises = load_iris(as_frame = True)['frame']

# Recoding the target column
df_exercises['target'].replace([0, 1, 2], ['setosa', 'versicolour', 'virginica'], inplace = True)

# Renaming columns
# DF.QUERY SEEMS TO NOT BE ABLE TO HANDLE COLUMN NAMES WITH PARENTHESES
# https://stackoverflow.com/questions/59422725/query-function-not-working-with-spaces-and-parenthesis-in-column-names
# https://github.com/pandas-dev/pandas/issues/49633
df_exercises.rename(
    columns = {
    "sepal length (cm)": "sepal length",
    "sepal width (cm)": "sepal width",
    "petal length (cm)": "petal_length",
    "petal width (cm)": "petal_width"
    },
    inplace = True
)

# Making sure the data looks ok
df_exercises

<br>
<br>
<br>

## `df.query`

`df.query` is one way to filter rows of a Pandas DataFrame.<br> <br>
As previously explained, you may prefer `df.query` if you have a background in SQL. You may also find it more intuitive or encounter it in other people's code. Finally, there could be some [efficiency gains](https://jakevdp.github.io/PythonDataScienceHandbook/03.12-performance-eval-and-query.html).<br><br>
You're probably already familiar with the most common way to filter rows, using `[...]`. You may have also learned about [`df.loc` and `df.iloc`](https://www.geeksforgeeks.org/difference-between-loc-and-iloc-in-pandas-dataframe/). This is an example of the most common way to filter rows:

In [ ]:
df[df['MEDITATE_A'] == "Yes"]

In this code, we are first selecting a column from the DataFrame (`df['MEDITATE_A']`). We're then specifying a condition to filter that column (`== "Yes"`). Finally, we're filtering the whole DataFrame based on that condition (`df[...]`).<br><br>

`df.query` is similar in that you need to specify the column and the filter that you want to apply to that column. However, the syntax is a bit different.

<br>
<br>

**Filtering on one condition:**

This is how you'd rewrite the example above using `df.query`:

In [ ]:
df.query('MEDITATE_A == "Yes"')

Notice four things:
- `query` is a method of Pandas DataFrames, so you use the syntax `df.query()`.
- The query goes in quotation marks. You can use `'` or `"`. Be mindful of whether you're including strings inside of the query.
- `query` returns a DataFrame. You can do all sorts of other operations on that DataFrame.
- If you want to modify the original DataFrame, you can use the argument `inplace = True`. You can also assign the resulting DataFrame to a new variable.

<br>
<br>

Using what we just learned, let's answer our first question--i.e., what percentage of American adults meditate?

In [ ]:
df.query('MEDITATE_A == "Yes"').shape[0] / df['MEDITATE_A'].dropna().shape[0]

This line of code includes two other methods besides `df.query`. You may or may not be familiar with those. Don't worry if you're not. They're not essential for this notebook. However, if you're curious, [`df.shape`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.shape.html) returns a tuple representing the dimensionality of the DataFrame and [`df.dropna`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html) removes missing values. What we're doing is getting the number of rows for people who meditate (`df.query('MEDITATE_A == "Yes"').shape[0]`) and diving them by the number of rows in the DataFrame where `MEDITATE_A` is not missing (`df['MEDITATE_A'].dropna().shape[0]`). That gives us an estimate of the percentage of American adults who meditate... about 20%!

<br>
<br>

**Exercise 1:**

How many rows does the Iris DataFrame has for setosa? Use `df.query` to get only the rows for which the `target` is "setosa".

Remember that the DataFrame for the exercises is called `df_exercises`.

<br>
<br>

**Filtering on two conditions:**

In [ ]:
df.query('MEDITATE_A == "Yes" or YOGAMED_A == "Yes"').shape[0] / df[['MEDITATE_A', 'YOGAMED_A']].dropna(how = 'all').shape[0]

The percentage of Americans who meditate is actually about 23% if we included people who meditate as part of yoga... but let's not worry about that!<br><br>

Notice that this code uses `or`. You can also use `|`, `and`, and `&`.

An equivalent way to filter rows in the code above is `df[(df['MEDITATE_A'] == "Yes") | (df['YOGAMED_A'] == "Yes")]`.

<br>
<br>

**Filtering based on a numerical column:**

Now let's answer our second questions: what percentage of college-age American adults meditate?

In [ ]:
df.query('AGEP_A < 22')

Notice that for numerical columns, you don't use quotation marks inside the query. It's `'AGEP_A < 22'`, not `'AGEP_A < "22"'`.

Other operators such as `>`, `>=`, `<=`, and `!=` are also supported.

<br>
<br>

An equivalent way to filter rows in the code above is `df[df['AGEP_A'] < 22]`.

<br>
<br>

The code above didn't actually answer our question--it simply returned a DataFrame with the rows where `AGEP_A < 22`. Now let's use some more Pandas to get the answer...

In [ ]:
df.query('AGEP_A < 22')['MEDITATE_A'].value_counts(normalize = True)

About 17% of American adults under the age of 22 meditate. Seems less than in the overall population!

In this code, we're selecting the column `MEDITATE_A` from the resulting DataFrame (`df.query('AGEP_A < 22')['MEDITATE_A']`). We're then using [`value_counts`](https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html) to get the proportion of rows for each unique value. Notice that we specified `normalize = True`. If you don't specify that, you get counts instead of proportions.

<br>
<br>

**Exercise 2:**

How many rows does the Iris DataFrame has for setosa with a petal width smaller than or equal to 0.2? Use `df.query` to get only the rows for which the `target` is "setosa" *and* the `petal_width` is equal to or less than 0.2.

<br>
<br>

**Filtering using `in ['element_1', ..., 'element_n']`:**

Let's move unto our third question: what percentage of Americans in large metropolitan areas meditate?

In [ ]:
# As a reminder, URBRRL is the 2013 NCHS Urban-Rural Classification Scheme for Counties
df.query('URBRRL in ["Large central metro", "Large fringe metro"]')

Notice the syntax here: `'column_name in [list_of_values]'`.

An alternative way to write this code is `df[df['URBRRL'].isin(["Large central metro", "Large fringe metro"])]`.

<br>
<br>

And the answer to our question is...

In [ ]:
df.query('URBRRL in ["Large central metro", "Large fringe metro"]')['MEDITATE_A'].value_counts(normalize = True)

About 22% of American adults in large metropolitan areas meditate. Seems a bit higher than in the overall population!

<br>
<br>

**Exercise 3:**

How many rows does the Iris DataFrame has for setosa or virginica? Use `df.query` to get only the rows for which the `target` is "setosa" *or* "virginica".

Bonus: try implementing this in three different ways!

<br>
<br>
<br>

## `df.eval`

`df.eval` is one way to describe operations over columns of a Pandas DataFrame.

As previously explained, you may find `df.eval` more intuitive or easier to understand. You may also encounter this syntax in other people's code. Finally, there may be some [efficiency gains](https://jakevdp.github.io/PythonDataScienceHandbook/03.12-performance-eval-and-query.html).

<br>
<br>

**Finding the sum of two columns:**

Before being familiar with `df.eval`, how would you find the sum of two columns in a DataFrame? You'd probably do this:

In [ ]:
df['PCNTADLT_A'] + df['PCNTKIDS_A']

Good news! Now you have another way to go about it:

In [ ]:
df.eval('PCNTADLT_A + PCNTKIDS_A')

Notice that the expression to evaluate goes inside quotation marks. Similarly to `query`, you can use both `'` and `"`.

This code uses `+`. Other operators such as `-`, `*`, and `/` are also supported (see [documentation of `pd.eval`](https://pandas.pydata.org/docs/reference/api/pandas.eval.html#pandas.eval) for all supported operations).

<br>
<br>

**Creating new column based on the sum of two existing columns:**

In [ ]:
df.eval('count_people_household = PCNTADLT_A + PCNTKIDS_A')

Notice that this code assigns the result of the evaluation to a new column (`count_people_household`) within the quotation marks. By default, the column is located at the end of the DataFrame.

<br>
<br>

**Modifying DataFrame inplace:**

We're going to use `df.eval` to answer our final question: does the percentage of Americans who meditate depend on the number of people living in the household? Before getting to that, however, it's worth pointing out that we haven't modified `df` so far:

In [ ]:
df

See? By default, `df.eval` doesn't modify the dataset. You need to specify `inplace = True`:

In [ ]:
df.eval('count_people_household = PCNTADLT_A + PCNTKIDS_A', inplace = True)
df

Similarly to `query`, you can modify the DataFrame inplace by using the argument `inplace = True`.

An alternative way to write the code above would be `df['count_people_household'] = df['PCNTADLT_A'] + df['PCNTKIDS_A']`.

<br>
<br>

We have our DataFrame. Now let's use a bit more Python and Pandas to answer our question:

In [ ]:
for value in np.sort(df['count_people_household'].dropna().unique()):
  print(value, ":")
  print(df.query('count_people_household == ' + str(value))['MEDITATE_A'].value_counts(normalize = True), '\n')

The percentage of American adults who meditate doesn't seem to be related to the number of people living in the household!

This code has a lot of elements, so let's unpack it. We're iterating over the values of `count_people_household` to filter the DataFrame and output the results:
- `df['count_people_household']` selects `count_people_household` from the DataFrame.
- `.dropna()` keeps only the rows where `count_people_household` is not missing.
- `.unique()` returns only the unique values of `count_people_household`.
- `np.sort` sorts the result of `df['count_people_household'].dropna().unique()` so that it's easier to interpret the results.
- `print(value, ":")` helps us print each value in a way that makes our for-loop readable.
- `str(value)` turns the numeric value into a string and we then use that string within `df.query('count_people_household == ' + str(value))` to filter the rows of the DataFrame where `count_people_household` takes on that value.
- We then select the column `MEDITATE_A` and get proportions using the same method than before (`['MEDITATE_A'].value_counts(normalize = True)`).
- Finally, we add `'\n'` to the the `print` statement to make the results of the for-loop more readable.

<br>
<br>

**Exercise 4:**

What's the ratio of petal length by petal width? Use `df.eval` to create a new column called `my_division`, which is the result of dividing `petal_length` by `petal_width`. Modify `df_exercises` inplace.

<br>
<br>
<br>

## Resources to continue learning:
- Pandas documentation
  - [df.query](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.query.html)
  - [df.eval](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.eval.html)
- Free tutorials
  - [df.query](https://www.askpython.com/python-modules/pandas/query-pandas-dataframe)
  - [pd.eval](https://www.askpython.com/python-modules/pandas/pandas-eval)

<br>
<br>
<br>

## Exercises to continue practicing:

Some slightly more challenging exercises.

Remember that the DataFrame for the exercises is called `df_exercises`.

<br>
<br>

**Exercise 5:**

How many rows does the Iris DataFrame has for flowers with sepal length equal to 5.1? Use `df.query` to filter the rows for which the column `sepal length` is equal to 5.1. You may want to take a look at the [documentation of `df.query`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.query.html) to figure out how to do this.

**Exercise 6:**

Re-write this code using `df.query`:
```
df_exercises[(df_exercises['petal_length'] == 1.4) | (df_exercises['petal_width'] == 1.1)]
```

**Exercise 7:**

Re-write this code using `df.query`:
```
df_exercises[df_exercises['petal_width'].between(0.2, 2.3)]
```
Bonus: try implementing this in two different ways!

**Exercise 8:**

Re-write this code using `df.eval`:
```
df_exercises['new_col'] = df_exercises['petal_length'] + df_exercises['petal_width']
```

**Exercise 9:**

Re-write this code using `df.eval` **only once**:
```
df_exercises['new_col_2'] = df_exercises['petal_length'] * 2
df_exercises['new_col_3'] = df_exercises['sepal length'] + df_exercises['sepal width']
```
You may want to take a look at the [documentation of `df.eval`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.eval.html).

<br>
<br>
<br>

## Solutions to the exercises:

**Exercise 1:**

In [ ]:
df_exercises.query('target == "setosa"')

**Exercise 2:**

In [ ]:
df_exercises.query('target == "setosa" and petal_width <= 0.2')

**Exercise 3:**

In [ ]:
df_exercises.query("target in ['setosa', 'virginica']")

In [ ]:
df_exercises.query("target == 'setosa' or target == 'virginica'")

In [ ]:
df_exercises.query("target != 'versicolour'")

**Exercise 4:**

In [ ]:
df_exercises.eval('my_division = petal_length / petal_width', inplace = True)

**Exercise 5:**

In [ ]:
df_exercises.query('`sepal length` == 5.1')

**Exercise 6:**

In [ ]:
df_exercises.query('petal_length == 1.4 or petal_width == 1.1')

In [ ]:
df_exercises.query('petal_length == 1.4 | petal_width == 1.1')

**Exercise 7:**

In [ ]:
df_exercises.query('0.2 <= petal_width <= 2.3')

In [ ]:
df_exercises.query('petal_width >= 0.2 and petal_width <= 2.3')

**Exercise 8:**

In [ ]:
df_exercises.eval('new_col = petal_length + petal_width', inplace = True)

**Exercise 9:**

In [ ]:
df_exercises.eval(
    """
    new_col_2 = petal_length * 2
    new_col_3 = `sepal length` + `sepal width`
    """,
    inplace = True
)